In [ ]:
import numpy as np
import sympy
import qsymm

sympy.init_printing(print_builtin=True)
np.set_printoptions(precision=2, suppress=True, linewidth=150)

# Find symmetries

Define Rashba-Hamiltonian

In [ ]:
ham1 = ("hbar^2 / (2 * m) * (k_x**2 + k_y**2 + k_z**2) * eye(2) +" +
        "alpha * sigma_x * k_x + alpha * sigma_y * k_y + alpha * sigma_z * k_z")
# Convert to standard monomials form
H1 = qsymm.Model(ham1)
H1.tosympy()

Define `cubic_group` as the set of candidates for point group symmetries.

In [ ]:
cubic_group = qsymm.groups.cubic()

Find point group symmetries, and discrete onsite and continuous symmetries.

In [ ]:
sg, cg = qsymm.symmetries(H1, cubic_group)
print(len(sg))
print(cg)

Print the names of the group elements. It includes time reversal, but not inversion.

In [ ]:
[display(g) for g in sg];

Detailed printout in LateX format.

In [ ]:
from IPython.display import display, Math
[display(Math(qsymm.groups.pretty_print_pge(g, latex=True, full=True))) for g in sg];

Check that this is the same as the full cubic group without inversion plus TR.

In [ ]:
C4 = qsymm.PointGroupElement(np.array([[1, 0, 0],
                                       [0, 0, 1],
                                       [0, -1, 0]]), False, False, None)
C3 = qsymm.PointGroupElement(np.array([[0, 0, 1],
                                       [1, 0, 0],
                                       [0, 1, 0]]), False, False, None)
TR = qsymm.PointGroupElement(np.eye(3), True, False, None)

set(sg) == qsymm.groups.generate_group({C4, C3, TR})

Add degeneracy, adds an `SU(2)` continuous symmetry group.

In [ ]:
ham2 = "kron(eye(2), " + ham1 + ")"
print(ham2)
# Convert to standard monomials form
H2 = qsymm.Model(ham2)
display(H2.tosympy())
sg, cg = qsymm.symmetries(H2, cubic_group)
print(len(sg))
[display(g) for g in cg];

Add hole degrees of freedom instead, only adds `U(1)` charge conservation, as there is no pairing.

In [ ]:
ham3 = "kron(sigma_z, " + ham1 + ")"
print(ham3)
# Convert to standard monomials form
H3 = qsymm.Model(ham3)
display(H3.tosympy())
sg, cg = qsymm.symmetries(H3, cubic_group)
print(len(sg))
[display(g) for g in cg];

Define Rashba-Hamiltonian with J = 3/2

In [ ]:
J_x, J_y, J_z = qsymm.groups.spin_matrices(3/2)
ham32 = ("hbar^2 / (2 * m) * (k_x**2 + k_y**2 + k_z**2) * eye(4) +" +
        "alpha * J_x * k_x + alpha * J_y * k_y + alpha * J_z * k_z")
# Convert to standard monomials form
H1 = qsymm.Model(ham32, locals=dict(J_x=J_x, J_y=J_y, J_z=J_z))
H1.tosympy(nsimplify=True)

Define `cubic_group` as the set of candidates for point group symmetries.

In [ ]:
sg, cg = qsymm.symmetries(H1, cubic_group)
print(len(sg))

Check taht this is the same as the full cubic group without inversion plus TR.

In [ ]:
set(sg) == qsymm.groups.generate_group({C4, C3, TR})

# Bloch Hamiltonian

Define hexagonal point group in 2D

In [ ]:
hex_group_2D = qsymm.groups.hexagonal()
print(len(hex_group_2D))

Single band nearest neighbor hopping in triangular lattice

In [ ]:
ham6 = 'm * (cos(k_x) + cos(1/2*k_x + sqrt(3)/2*k_y) + cos(-1/2*k_x + sqrt(3)/2*k_y))'
display(qsymm.sympify(ham6))
H6 = qsymm.Model(ham6, momenta=[0, 1])
sg, cg = qsymm.symmetries(H6, hex_group_2D)
print(len(sg))
print(set(sg) == qsymm.groups.hexagonal(ph=False))
print(cg)

Add spin and Rashba term

In [ ]:
ham62 = 'eye(2) * (' + ham6 + ') +'
ham62 += 'alpha * (sin(k_x) * sigma_x + sin(1/2*k_x + sqrt(3)/2*k_y) * (1/2*sigma_x + sqrt(3)/2*sigma_y) +'
ham62 += 'sin(-1/2*k_x + sqrt(3)/2*k_y) * (-1/2*sigma_x + sqrt(3)/2*sigma_y))'
print(ham62)
display(qsymm.sympify(ham62))
H62 = qsymm.Model(ham62, momenta=[0, 1])
sg, cg = qsymm.symmetries(H62, hex_group_2D)
print(len(sg))
print(set(sg) == qsymm.groups.hexagonal(ph=False))
print(cg)

Add degeneracy

In [ ]:
ham63 = 'kron(eye(2), ' + ham62 + ')'
display(qsymm.sympify(ham63))
H63 = qsymm.Model(ham63, momenta=[0, 1])
sg, cg = qsymm.symmetries(H63, hex_group_2D)
print(len(sg))
print(set(sg) == qsymm.groups.hexagonal(ph=False))
[display(g) for g in cg];

Add hole degrees of freedom

In [ ]:
ham64 = 'kron(sigma_z, ' + ham62 + ')'
print(ham64)
display(qsymm.sympify(ham64))
H64 = qsymm.Model(ham64, momenta=[0, 1])
sg, cg = qsymm.symmetries(H64, hex_group_2D)
print(len(sg))
print(set(sg) == hex_group_2D)
[display(g) for g in cg];

# Continuous Rotation Symmetry

Rashba Hamiltonian actually has full rotation invariance.

In [ ]:
display(qsymm.sympify(ham1))
pg, cg = qsymm.symmetries(H1, continuous_rotations=True, prettify=True)

In [ ]:
[display(g) for g in cg];

# More complicated examples
## k.p Hamiltonian

Use an 8x8 k.p Hamiltonian of zinc blende semiconductors.

In [ ]:
serialized_kp = 'Matrix([[hbar**2*k_x*gamma_0*k_x/(2*m_0)+hbar**2*k_y*gamma_0*k_y/(2*m_0)+hbar**2*k_z*gamma_0*k_z/(2*m_0)+E_0+E_v,0,-sqrt(2)*P*k_x/2-sqrt(2)*I*P*k_y/2,sqrt(6)*P*k_z/3,sqrt(6)*P*k_x/6-sqrt(6)*I*P*k_y/6,0,-sqrt(3)*P*k_z/3,-sqrt(3)*P*k_x/3+sqrt(3)*I*P*k_y/3],[0,hbar**2*k_x*gamma_0*k_x/(2*m_0)+hbar**2*k_y*gamma_0*k_y/(2*m_0)+hbar**2*k_z*gamma_0*k_z/(2*m_0)+E_0+E_v,0,-sqrt(6)*P*k_x/6-sqrt(6)*I*P*k_y/6,sqrt(6)*P*k_z/3,sqrt(2)*P*k_x/2-sqrt(2)*I*P*k_y/2,-sqrt(3)*P*k_x/3-sqrt(3)*I*P*k_y/3,sqrt(3)*P*k_z/3],[-sqrt(2)*k_x*P/2+sqrt(2)*I*k_y*P/2,0,-hbar**2*k_x*gamma_1*k_x/(2*m_0)-hbar**2*k_x*gamma_2*k_x/(2*m_0)-I*hbar**2*k_x*k_y/(2*m_0)-3*I*hbar**2*k_x*kappa*k_y/(2*m_0)-hbar**2*k_y*gamma_1*k_y/(2*m_0)-hbar**2*k_y*gamma_2*k_y/(2*m_0)+I*hbar**2*k_y*k_x/(2*m_0)+3*I*hbar**2*k_y*kappa*k_x/(2*m_0)-hbar**2*k_z*gamma_1*k_z/(2*m_0)+hbar**2*k_z*gamma_2*k_z/m_0+E_v,sqrt(3)*hbar**2*k_x*gamma_3*k_z/(2*m_0)+sqrt(3)*hbar**2*k_x*k_z/(6*m_0)+sqrt(3)*hbar**2*k_x*kappa*k_z/(2*m_0)-sqrt(3)*I*hbar**2*k_y*gamma_3*k_z/(2*m_0)-sqrt(3)*I*hbar**2*k_y*k_z/(6*m_0)-sqrt(3)*I*hbar**2*k_y*kappa*k_z/(2*m_0)+sqrt(3)*hbar**2*k_z*gamma_3*k_x/(2*m_0)-sqrt(3)*I*hbar**2*k_z*gamma_3*k_y/(2*m_0)-sqrt(3)*hbar**2*k_z*k_x/(6*m_0)+sqrt(3)*I*hbar**2*k_z*k_y/(6*m_0)-sqrt(3)*hbar**2*k_z*kappa*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_z*kappa*k_y/(2*m_0),sqrt(3)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(3)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)-sqrt(3)*hbar**2*k_y*gamma_2*k_y/(2*m_0)-sqrt(3)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),0,-sqrt(6)*hbar**2*k_x*gamma_3*k_z/(4*m_0)-sqrt(6)*hbar**2*k_x*k_z/(12*m_0)-sqrt(6)*hbar**2*k_x*kappa*k_z/(4*m_0)+sqrt(6)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)+sqrt(6)*I*hbar**2*k_y*k_z/(12*m_0)+sqrt(6)*I*hbar**2*k_y*kappa*k_z/(4*m_0)-sqrt(6)*hbar**2*k_z*gamma_3*k_x/(4*m_0)+sqrt(6)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)+sqrt(6)*hbar**2*k_z*k_x/(12*m_0)-sqrt(6)*I*hbar**2*k_z*k_y/(12*m_0)+sqrt(6)*hbar**2*k_z*kappa*k_x/(4*m_0)-sqrt(6)*I*hbar**2*k_z*kappa*k_y/(4*m_0),-sqrt(6)*hbar**2*k_x*gamma_2*k_x/(2*m_0)+sqrt(6)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)+sqrt(6)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(6)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0)],[sqrt(6)*k_z*P/3,-sqrt(6)*k_x*P/6+sqrt(6)*I*k_y*P/6,sqrt(3)*hbar**2*k_x*gamma_3*k_z/(2*m_0)-sqrt(3)*hbar**2*k_x*k_z/(6*m_0)-sqrt(3)*hbar**2*k_x*kappa*k_z/(2*m_0)+sqrt(3)*I*hbar**2*k_y*gamma_3*k_z/(2*m_0)-sqrt(3)*I*hbar**2*k_y*k_z/(6*m_0)-sqrt(3)*I*hbar**2*k_y*kappa*k_z/(2*m_0)+sqrt(3)*hbar**2*k_z*gamma_3*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_z*gamma_3*k_y/(2*m_0)+sqrt(3)*hbar**2*k_z*k_x/(6*m_0)+sqrt(3)*I*hbar**2*k_z*k_y/(6*m_0)+sqrt(3)*hbar**2*k_z*kappa*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_z*kappa*k_y/(2*m_0),-hbar**2*k_x*gamma_1*k_x/(2*m_0)+hbar**2*k_x*gamma_2*k_x/(2*m_0)-I*hbar**2*k_x*k_y/(6*m_0)-I*hbar**2*k_x*kappa*k_y/(2*m_0)-hbar**2*k_y*gamma_1*k_y/(2*m_0)+hbar**2*k_y*gamma_2*k_y/(2*m_0)+I*hbar**2*k_y*k_x/(6*m_0)+I*hbar**2*k_y*kappa*k_x/(2*m_0)-hbar**2*k_z*gamma_1*k_z/(2*m_0)-hbar**2*k_z*gamma_2*k_z/m_0+E_v,hbar**2*k_x*k_z/(3*m_0)+hbar**2*k_x*kappa*k_z/m_0-I*hbar**2*k_y*k_z/(3*m_0)-I*hbar**2*k_y*kappa*k_z/m_0-hbar**2*k_z*k_x/(3*m_0)+I*hbar**2*k_z*k_y/(3*m_0)-hbar**2*k_z*kappa*k_x/m_0+I*hbar**2*k_z*kappa*k_y/m_0,sqrt(3)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(3)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)-sqrt(3)*hbar**2*k_y*gamma_2*k_y/(2*m_0)-sqrt(3)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),-sqrt(2)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(2)*I*hbar**2*k_x*k_y/(6*m_0)-sqrt(2)*I*hbar**2*k_x*kappa*k_y/(2*m_0)-sqrt(2)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(2)*I*hbar**2*k_y*k_x/(6*m_0)+sqrt(2)*I*hbar**2*k_y*kappa*k_x/(2*m_0)+sqrt(2)*hbar**2*k_z*gamma_2*k_z/m_0,3*sqrt(2)*hbar**2*k_x*gamma_3*k_z/(4*m_0)-sqrt(2)*hbar**2*k_x*k_z/(12*m_0)-sqrt(2)*hbar**2*k_x*kappa*k_z/(4*m_0)-3*sqrt(2)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)+sqrt(2)*I*hbar**2*k_y*k_z/(12*m_0)+sqrt(2)*I*hbar**2*k_y*kappa*k_z/(4*m_0)+3*sqrt(2)*hbar**2*k_z*gamma_3*k_x/(4*m_0)-3*sqrt(2)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)+sqrt(2)*hbar**2*k_z*k_x/(12*m_0)-sqrt(2)*I*hbar**2*k_z*k_y/(12*m_0)+sqrt(2)*hbar**2*k_z*kappa*k_x/(4*m_0)-sqrt(2)*I*hbar**2*k_z*kappa*k_y/(4*m_0)],[sqrt(6)*k_x*P/6+sqrt(6)*I*k_y*P/6,sqrt(6)*k_z*P/3,sqrt(3)*hbar**2*k_x*gamma_2*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)-sqrt(3)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(3)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),-hbar**2*k_x*k_z/(3*m_0)-hbar**2*k_x*kappa*k_z/m_0-I*hbar**2*k_y*k_z/(3*m_0)-I*hbar**2*k_y*kappa*k_z/m_0+hbar**2*k_z*k_x/(3*m_0)+I*hbar**2*k_z*k_y/(3*m_0)+hbar**2*k_z*kappa*k_x/m_0+I*hbar**2*k_z*kappa*k_y/m_0,-hbar**2*k_x*gamma_1*k_x/(2*m_0)+hbar**2*k_x*gamma_2*k_x/(2*m_0)+I*hbar**2*k_x*k_y/(6*m_0)+I*hbar**2*k_x*kappa*k_y/(2*m_0)-hbar**2*k_y*gamma_1*k_y/(2*m_0)+hbar**2*k_y*gamma_2*k_y/(2*m_0)-I*hbar**2*k_y*k_x/(6*m_0)-I*hbar**2*k_y*kappa*k_x/(2*m_0)-hbar**2*k_z*gamma_1*k_z/(2*m_0)-hbar**2*k_z*gamma_2*k_z/m_0+E_v,-sqrt(3)*hbar**2*k_x*gamma_3*k_z/(2*m_0)+sqrt(3)*hbar**2*k_x*k_z/(6*m_0)+sqrt(3)*hbar**2*k_x*kappa*k_z/(2*m_0)+sqrt(3)*I*hbar**2*k_y*gamma_3*k_z/(2*m_0)-sqrt(3)*I*hbar**2*k_y*k_z/(6*m_0)-sqrt(3)*I*hbar**2*k_y*kappa*k_z/(2*m_0)-sqrt(3)*hbar**2*k_z*gamma_3*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_z*gamma_3*k_y/(2*m_0)-sqrt(3)*hbar**2*k_z*k_x/(6*m_0)+sqrt(3)*I*hbar**2*k_z*k_y/(6*m_0)-sqrt(3)*hbar**2*k_z*kappa*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_z*kappa*k_y/(2*m_0),3*sqrt(2)*hbar**2*k_x*gamma_3*k_z/(4*m_0)-sqrt(2)*hbar**2*k_x*k_z/(12*m_0)-sqrt(2)*hbar**2*k_x*kappa*k_z/(4*m_0)+3*sqrt(2)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)-sqrt(2)*I*hbar**2*k_y*k_z/(12*m_0)-sqrt(2)*I*hbar**2*k_y*kappa*k_z/(4*m_0)+3*sqrt(2)*hbar**2*k_z*gamma_3*k_x/(4*m_0)+3*sqrt(2)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)+sqrt(2)*hbar**2*k_z*k_x/(12*m_0)+sqrt(2)*I*hbar**2*k_z*k_y/(12*m_0)+sqrt(2)*hbar**2*k_z*kappa*k_x/(4*m_0)+sqrt(2)*I*hbar**2*k_z*kappa*k_y/(4*m_0),sqrt(2)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(2)*I*hbar**2*k_x*k_y/(6*m_0)-sqrt(2)*I*hbar**2*k_x*kappa*k_y/(2*m_0)+sqrt(2)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(2)*I*hbar**2*k_y*k_x/(6*m_0)+sqrt(2)*I*hbar**2*k_y*kappa*k_x/(2*m_0)-sqrt(2)*hbar**2*k_z*gamma_2*k_z/m_0],[0,sqrt(2)*k_x*P/2+sqrt(2)*I*k_y*P/2,0,sqrt(3)*hbar**2*k_x*gamma_2*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)-sqrt(3)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(3)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),-sqrt(3)*hbar**2*k_x*gamma_3*k_z/(2*m_0)-sqrt(3)*hbar**2*k_x*k_z/(6*m_0)-sqrt(3)*hbar**2*k_x*kappa*k_z/(2*m_0)-sqrt(3)*I*hbar**2*k_y*gamma_3*k_z/(2*m_0)-sqrt(3)*I*hbar**2*k_y*k_z/(6*m_0)-sqrt(3)*I*hbar**2*k_y*kappa*k_z/(2*m_0)-sqrt(3)*hbar**2*k_z*gamma_3*k_x/(2*m_0)-sqrt(3)*I*hbar**2*k_z*gamma_3*k_y/(2*m_0)+sqrt(3)*hbar**2*k_z*k_x/(6*m_0)+sqrt(3)*I*hbar**2*k_z*k_y/(6*m_0)+sqrt(3)*hbar**2*k_z*kappa*k_x/(2*m_0)+sqrt(3)*I*hbar**2*k_z*kappa*k_y/(2*m_0),-hbar**2*k_x*gamma_1*k_x/(2*m_0)-hbar**2*k_x*gamma_2*k_x/(2*m_0)+I*hbar**2*k_x*k_y/(2*m_0)+3*I*hbar**2*k_x*kappa*k_y/(2*m_0)-hbar**2*k_y*gamma_1*k_y/(2*m_0)-hbar**2*k_y*gamma_2*k_y/(2*m_0)-I*hbar**2*k_y*k_x/(2*m_0)-3*I*hbar**2*k_y*kappa*k_x/(2*m_0)-hbar**2*k_z*gamma_1*k_z/(2*m_0)+hbar**2*k_z*gamma_2*k_z/m_0+E_v,sqrt(6)*hbar**2*k_x*gamma_2*k_x/(2*m_0)+sqrt(6)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)-sqrt(6)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(6)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),-sqrt(6)*hbar**2*k_x*gamma_3*k_z/(4*m_0)-sqrt(6)*hbar**2*k_x*k_z/(12*m_0)-sqrt(6)*hbar**2*k_x*kappa*k_z/(4*m_0)-sqrt(6)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)-sqrt(6)*I*hbar**2*k_y*k_z/(12*m_0)-sqrt(6)*I*hbar**2*k_y*kappa*k_z/(4*m_0)-sqrt(6)*hbar**2*k_z*gamma_3*k_x/(4*m_0)-sqrt(6)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)+sqrt(6)*hbar**2*k_z*k_x/(12*m_0)+sqrt(6)*I*hbar**2*k_z*k_y/(12*m_0)+sqrt(6)*hbar**2*k_z*kappa*k_x/(4*m_0)+sqrt(6)*I*hbar**2*k_z*kappa*k_y/(4*m_0)],[-sqrt(3)*k_z*P/3,-sqrt(3)*k_x*P/3+sqrt(3)*I*k_y*P/3,-sqrt(6)*hbar**2*k_x*gamma_3*k_z/(4*m_0)+sqrt(6)*hbar**2*k_x*k_z/(12*m_0)+sqrt(6)*hbar**2*k_x*kappa*k_z/(4*m_0)-sqrt(6)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)+sqrt(6)*I*hbar**2*k_y*k_z/(12*m_0)+sqrt(6)*I*hbar**2*k_y*kappa*k_z/(4*m_0)-sqrt(6)*hbar**2*k_z*gamma_3*k_x/(4*m_0)-sqrt(6)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)-sqrt(6)*hbar**2*k_z*k_x/(12*m_0)-sqrt(6)*I*hbar**2*k_z*k_y/(12*m_0)-sqrt(6)*hbar**2*k_z*kappa*k_x/(4*m_0)-sqrt(6)*I*hbar**2*k_z*kappa*k_y/(4*m_0),-sqrt(2)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(2)*I*hbar**2*k_x*k_y/(6*m_0)-sqrt(2)*I*hbar**2*k_x*kappa*k_y/(2*m_0)-sqrt(2)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(2)*I*hbar**2*k_y*k_x/(6*m_0)+sqrt(2)*I*hbar**2*k_y*kappa*k_x/(2*m_0)+sqrt(2)*hbar**2*k_z*gamma_2*k_z/m_0,3*sqrt(2)*hbar**2*k_x*gamma_3*k_z/(4*m_0)+sqrt(2)*hbar**2*k_x*k_z/(12*m_0)+sqrt(2)*hbar**2*k_x*kappa*k_z/(4*m_0)-3*sqrt(2)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)-sqrt(2)*I*hbar**2*k_y*k_z/(12*m_0)-sqrt(2)*I*hbar**2*k_y*kappa*k_z/(4*m_0)+3*sqrt(2)*hbar**2*k_z*gamma_3*k_x/(4*m_0)-3*sqrt(2)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)-sqrt(2)*hbar**2*k_z*k_x/(12*m_0)+sqrt(2)*I*hbar**2*k_z*k_y/(12*m_0)-sqrt(2)*hbar**2*k_z*kappa*k_x/(4*m_0)+sqrt(2)*I*hbar**2*k_z*kappa*k_y/(4*m_0),sqrt(6)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(6)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)-sqrt(6)*hbar**2*k_y*gamma_2*k_y/(2*m_0)-sqrt(6)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),-hbar**2*k_x*gamma_1*k_x/(2*m_0)-I*hbar**2*k_x*k_y/(3*m_0)-I*hbar**2*k_x*kappa*k_y/m_0-hbar**2*k_y*gamma_1*k_y/(2*m_0)+I*hbar**2*k_y*k_x/(3*m_0)+I*hbar**2*k_y*kappa*k_x/m_0-hbar**2*k_z*gamma_1*k_z/(2*m_0)-Delta+E_v,hbar**2*k_x*k_z/(3*m_0)+hbar**2*k_x*kappa*k_z/m_0-I*hbar**2*k_y*k_z/(3*m_0)-I*hbar**2*k_y*kappa*k_z/m_0-hbar**2*k_z*k_x/(3*m_0)+I*hbar**2*k_z*k_y/(3*m_0)-hbar**2*k_z*kappa*k_x/m_0+I*hbar**2*k_z*kappa*k_y/m_0],[-sqrt(3)*k_x*P/3-sqrt(3)*I*k_y*P/3,sqrt(3)*k_z*P/3,-sqrt(6)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(6)*I*hbar**2*k_x*gamma_3*k_y/(2*m_0)+sqrt(6)*hbar**2*k_y*gamma_2*k_y/(2*m_0)-sqrt(6)*I*hbar**2*k_y*gamma_3*k_x/(2*m_0),3*sqrt(2)*hbar**2*k_x*gamma_3*k_z/(4*m_0)+sqrt(2)*hbar**2*k_x*k_z/(12*m_0)+sqrt(2)*hbar**2*k_x*kappa*k_z/(4*m_0)+3*sqrt(2)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)+sqrt(2)*I*hbar**2*k_y*k_z/(12*m_0)+sqrt(2)*I*hbar**2*k_y*kappa*k_z/(4*m_0)+3*sqrt(2)*hbar**2*k_z*gamma_3*k_x/(4*m_0)+3*sqrt(2)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)-sqrt(2)*hbar**2*k_z*k_x/(12*m_0)-sqrt(2)*I*hbar**2*k_z*k_y/(12*m_0)-sqrt(2)*hbar**2*k_z*kappa*k_x/(4*m_0)-sqrt(2)*I*hbar**2*k_z*kappa*k_y/(4*m_0),sqrt(2)*hbar**2*k_x*gamma_2*k_x/(2*m_0)-sqrt(2)*I*hbar**2*k_x*k_y/(6*m_0)-sqrt(2)*I*hbar**2*k_x*kappa*k_y/(2*m_0)+sqrt(2)*hbar**2*k_y*gamma_2*k_y/(2*m_0)+sqrt(2)*I*hbar**2*k_y*k_x/(6*m_0)+sqrt(2)*I*hbar**2*k_y*kappa*k_x/(2*m_0)-sqrt(2)*hbar**2*k_z*gamma_2*k_z/m_0,-sqrt(6)*hbar**2*k_x*gamma_3*k_z/(4*m_0)+sqrt(6)*hbar**2*k_x*k_z/(12*m_0)+sqrt(6)*hbar**2*k_x*kappa*k_z/(4*m_0)+sqrt(6)*I*hbar**2*k_y*gamma_3*k_z/(4*m_0)-sqrt(6)*I*hbar**2*k_y*k_z/(12*m_0)-sqrt(6)*I*hbar**2*k_y*kappa*k_z/(4*m_0)-sqrt(6)*hbar**2*k_z*gamma_3*k_x/(4*m_0)+sqrt(6)*I*hbar**2*k_z*gamma_3*k_y/(4*m_0)-sqrt(6)*hbar**2*k_z*k_x/(12*m_0)+sqrt(6)*I*hbar**2*k_z*k_y/(12*m_0)-sqrt(6)*hbar**2*k_z*kappa*k_x/(4*m_0)+sqrt(6)*I*hbar**2*k_z*kappa*k_y/(4*m_0),-hbar**2*k_x*k_z/(3*m_0)-hbar**2*k_x*kappa*k_z/m_0-I*hbar**2*k_y*k_z/(3*m_0)-I*hbar**2*k_y*kappa*k_z/m_0+hbar**2*k_z*k_x/(3*m_0)+I*hbar**2*k_z*k_y/(3*m_0)+hbar**2*k_z*kappa*k_x/m_0+I*hbar**2*k_z*kappa*k_y/m_0,-hbar**2*k_x*gamma_1*k_x/(2*m_0)+I*hbar**2*k_x*k_y/(3*m_0)+I*hbar**2*k_x*kappa*k_y/m_0-hbar**2*k_y*gamma_1*k_y/(2*m_0)-I*hbar**2*k_y*k_x/(3*m_0)-I*hbar**2*k_y*kappa*k_x/m_0-hbar**2*k_z*gamma_1*k_z/(2*m_0)-Delta+E_v]])'
display(qsymm.sympify(serialized_kp))
Hkp = qsymm.Model(serialized_kp)

Symmetry is full cubic symmetry and time reversal.

In [ ]:
sg, cg = qsymm.symmetries(Hkp, cubic_group)
print(len(sg))
[display(g) for g in cg];

Examples of how restricting parameters changes the symmetry group

In [ ]:
kp_examples = [
    {'hbar': 1},
    {'hbar': 1, 'P': 0, 'Delta': 0},
    {'hbar': 1, 'P': 0, 'Delta': 0, 'gamma_3': 0},
    {'hbar': 1, 'gamma_2': 'gamma_1', 'gamma_3': 'gamma_1', 'k_z': 0},
    {'hbar': 1, 'P': 0, 'Delta': 0, 'gamma_3': 0},
    {'hbar': 1, 'Delta': 0, 'k_z': 0},    
]

for subs in kp_examples:
    print(subs)
    Hkp = qsymm.Model(serialized_kp, locals=subs)
    %time sg, cg = qsymm.symmetries(Hkp, cubic_group)
    print(len(sg), len(cg))

In [ ]:
import itertools as it
import scipy.linalg as la
from qsymm.linalg import prop_to_id, allclose
from qsymm.groups import PointGroupElement, set_multiply, generate_group, _U_strict_eq

In [ ]:
def identity_power(g, Ps=None, double_group=False):
    # Append on-site U(1) symmetries to PG element to ensure that 
    # n'th power is exactly the identity.
    # If double_group=True, if the n'th power is 2\pi rotation,
    # -identity is chosen
    def find_order(r):
        dim = r.shape[0]
        rpower = r
        order = 1
        while not np.allclose(rpower, np.eye(dim)):
            rpower = r.dot(rpower)
            order += 1
        return order
    
    R = np.array(g.R).astype(float)
    n = find_order(R)
    sign = 1
    if double_group and n>1:
        if np.isclose(la.det(R), 1):
            # If not rotoinversion, its n'th power is 2pi rotation
            sign = -1
        elif find_order(-R) == n:
            # here we assume R is 3d rotoinversion
            # R^n is 2pi rotation
            sign = -1
            # if -R has a lower order (half of n) then 
            # R^n is a 4pi rotation or identity

    Un = np.linalg.matrix_power(g.U, n)

    if Ps is not None:
        t = np.zeros_like(g.U)
        for P in Ps:
            # reduce Un to one block of the block
            Unr = P[0].T.conj() @ Un @ P[0]
            prop_id, coeff = prop_to_id(Unr)
            assert prop_id
            # transform U by n'th root
            t += coeff ** (-1/n) * np.einsum('aij,akj->ik', P, P.conjugate())
    else:
        prop_id, coeff = prop_to_id(Un)
        assert prop_id, (g.U, Un)
        t = coeff ** (-1/n) * np.eye(Un.shape[0])

    t = t * sign ** (-1/n)
    g_new = PointGroupElement(g.R, g.conjugate, g.antisymmetry, U=g.U @ t, _U_eq=_U_strict_eq)
    assert allclose((g_new**n).U, sign * np.eye(g_new.U.shape[0]))
    return g_new, n, sign

In [ ]:
[identity_power(g, double_group=True) for g in sg if g.conjugate==False]

In [ ]:
cubic = qsymm.groups.cubic(tr=False, ph=False, generators=True)
gens = [g for g in sg if g in cubic]
gens

In [ ]:
def check_consistency(new_group, double_group):
    for g in new_group:
        _, n, sign = identity_power(g, double_group=double_group)
        if n % 2 == 0 and not allclose((g**n).U, sign * np.eye(g.U.shape[0])):
            # print(g, (g**n).U, sign, n)
            return False
    return True

def U_double_eq(U1, U2):
    if allclose(U1, U2):
        return True
    elif allclose(U1, -U2):
        return False
    else:
        raise ValueError("Improper phase fixing detected in unitary parts of symmetry operators.")

def U_single_eq(U1, U2):
    if allclose(U1, U2):
        return True
    else:
        raise ValueError("Improper phase fixing detected in unitary parts of symmetry operators.")


def fix_pg_phases(generators, double_group=False):
    """
    Fix phases of unitaries such that the (double) point group generated by generators
    forms a true representation.
    """
    if any(g.conjugate for g in generators) or any(g.antisymmetry for g in generators):
        raise ValueError('Only unitary point groups are supported. Make sure `generators` '
                         'do not contain any elements with conjugate=True or antisymmetry=True.')
    gens_orders = [identity_power(g, double_group=double_group) for g in generators]
    
    n_group = len(generate_group(generators))

    fixes = []
    
    U_eq = U_double_eq if double_group else U_single_eq

    for phases in it.product(*[range(n) for _, n, _ in gens_orders]):
        new_gens = []
        for (g, n, _), phase in zip(gens_orders, phases):
            phase = 2 * np.pi * phase / n
            g_new = PointGroupElement(g.R, U=np.exp(1j * phase) * g.U, _U_eq=U_eq)
            new_gens.append(g_new)
        try:
            # This will raise an error if not a single/double representation
            new_group = qsymm.groups.generate_group(new_gens)
        except ValueError:
            continue
        print(len(new_group))
        if ((double_group and not len(new_group) == 2 * n_group)
            or (not double_group and not len(new_group) == n_group)):
            continue
        # Check that the phase choices are consistent
        if check_consistency(new_group, double_group):
            # return new_gens
            fixes.append(new_gens)
            # print(new_gens)
            print(phases)
    return fixes[0]

In [ ]:
fix_pg_phases(gens, double_group=True)

In [ ]:
_U_strict_eq.cache_info()

In [ ]:
def conjugate_classes(group):
    rest = set(group)
    conjugate_classes = dict()
    while rest:
        g = rest.pop()
        conjugates = {h * g * h.inv() for h in group}
        conjugate_classes[g] = conjugates
        rest -= conjugates
    return conjugate_classes

In [ ]:
ccs = conjugate_classes(qsymm.groups.generate_group(gens))

In [ ]:
[(key, len(val)) for key, val in ccs.items()]

In [ ]:
def character_table(gens, double_group=False):
    """
    Find character table of group described by gens.
    """
    fixed_gens = fix_pg_phases(gens, double_group)
    fixed_group = generate_group(fixed_gens)
    ccs = conjugate_classes(fixed_group)
    charater_table = [(len(cc), rep, np.round(np.trace(rep.U), 3)) for rep, cc in ccs.items()]
    return charater_table

In [ ]:
character_table(gens, double_group=True)

In [ ]:
def fix_pg_phases(generators, double_group=False):
    """
    Fix phases of unitaries such that the (double) point group generated by generators
    forms a true representation.
    """
    if any(g.conjugate for g in generators) or any(g.antisymmetry for g in generators):
        raise ValueError('Only unitary point groups are supported. Make sure `generators` '
                         'do not contain any elements with conjugate=True or antisymmetry=True.')
    gens_orders = [identity_power(g, double_group=double_group) for g in generators]

    fixes = []

    for phases in it.product(*[range(n) for _, n in gens_orders]):
        new_gens = []
        for (g, n), phase in zip(gens_orders, phases):
            phase = 2 * np.pi * phase / n
            g_new = PointGroupElement(g.R, U=np.exp(1j * phase) * g.U, _U_eq=_U_strict_eq)
            new_gens.append(g_new)
        new_group = qsymm.groups.generate_group(new_gens)
        if len(new_group) > 96:
            continue
        # Check that the phase choices are consistent
        # Brute force
        for g, h in it.product(new_group, repeat=2):
            gh = g * h
            gh_new = [g for g in new_group if g == gh][0]
            if not (np.allclose(gh_new.U, gh.U)
                    or (double_group and np.allclose(gh_new.U, -gh.U))):
                # If inconsistent, break
                break
        # If didn't break, we found a consistent phase choice
        else:
            # return new_gens
            fixes.append(new_gens)
            # print(new_gens)
            # print(phases)
    return fixes

In [ ]:
def character_table(gens, double_group=False):
    """
    Find character table of group described by gens.
    """
    for fixed_gens in fix_pg_phases(gens, double_group):
        fixed_group = generate_group(fixed_gens)
        ccs = conjugate_classes(fixed_group)
        charater_table = [(len(cc), rep, np.round(np.trace(rep.U), 3)) for rep, cc in ccs.items()]
        display(charater_table)

In [ ]:
character_table(gens, double_group=True)

In [ ]:
np.dot(ta.array(np.eye(2)), ta.array(np.eye(2)))

In [ ]:
import tinyarray as ta

In [ ]:
a = ta.array(1j * np.eye(2))

In [ ]:
type(a).mro()

In [ ]:
from qsymm.groups import rotation_to_angle, spin_rotation

def o3_to_su2(g):
    """
    Transform O(3) spatial rotation to SU(2) x Z2 representation.
    """
    R = np.array(g.R)
    if np.isclose(la.det(R), 1):
        inv = 1
        n, theta = rotation_to_angle(R)
    else:
        inv = -1
        n, theta = rotation_to_angle(-R)
    R_new = la.block_diag(spin_rotation(theta * n, 1/2), [[inv]])
    return PointGroupElement(R_new, g.conjugate, g.antisymmetry, g.U, g._U_eq)

In [ ]:
[o3_to_su2(g) for g in gens]